In [1]:
print("Hello, world!")

Hello, world!


In [2]:
!nvidia-smi

Mon Nov 17 08:27:16 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   57C    P8             11W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [7]:
%%writefile vec_add.cu

Writing vec_add.cu


In [12]:
!ls
!cd sample_data/.
!ls

sample_data  vec_add.cu
sample_data  vec_add.cu


In [13]:
!mkdir test_dir
!cd test_dir/.
!pwd

/content


In [16]:
!ls
cd test_dir/ && touch test-cuda.cu

SyntaxError: invalid syntax (ipython-input-136439876.py, line 2)

In [ ]:
%%writefile vec_add.cu
#include <stdio.h>

__global__ void vecAdd(const float *a, const float *b, float *c, int n) {
    int i = blockDim.x * blockIdx.x + threadIdx.x;
    if (i < n) c[i] = a[i] + b[i];
}

int main() {
    int n = 1 << 20; // 1M elements
    size_t size = n * sizeof(float);

    float *h_a = (float*)malloc(size);
    float *h_b = (float*)malloc(size);
    float *h_c = (float*)malloc(size);

    for (int i = 0; i < n; ++i) {
        h_a[i] = 1.0;
        h_b[i] = 2.0;
    }

    float *d_a, *d_b, *d_c;
    cudaMalloc(&d_a, size);
    cudaMalloc(&d_b, size);
    cudaMalloc(&d_c, size);

    cudaMemcpy(d_a, h_a, size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_b, h_b, size, cudaMemcpyHostToDevice);

    int blockSize = 256;
    int numBlocks = (n + blockSize - 1) / blockSize;
    vecAdd<<<numBlocks, blockSize>>>(d_a, d_b, d_c, n);

    cudaMemcpy(h_c, d_c, size, cudaMemcpyDeviceToHost);

    printf("c[0] = %f\n", h_c[0]);
    printf("c[n-1] = %f\n", h_c[n-1]);

    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_c);
    free(h_a);
    free(h_b);
    free(h_c);
    return 0;
}

Overwriting vec_add.cu


In [20]:
!nvcc vec_add.cu -o vec_add
!./vec_add

c[0] = 0.000000
c[n-1] = 0.000000


In [29]:
!curl https://developer.download.nvidia.com/hpc-sdk/ubuntu/DEB-GPG-KEY-NVIDIA-HPC-SDK | sudo gpg --dearmor -o /usr/share/keyrings/nvidia-hpcsdk-archive-keyring.gpg

!echo 'deb [signed-by=/usr/share/keyrings/nvidia-hpcsdk-archive-keyring.gpg] https://developer.download.nvidia.com/hpc-sdk/ubuntu/amd64 /' !| sudo tee /etc/apt/sources.list.d/nvhpc.list

!sudo apt-get update -y
!sudo apt-get install -y nvhpc-25-

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1626  100  1626    0     0   2950      0 --:--:-- --:--:-- --:--:--  2950
deb [signed-by=/usr/share/keyrings/nvidia-hpcsdk-archive-keyring.gpg] https://developer.download.nvidia.com/hpc-sdk/ubuntu/amd64 / !
E: Malformed entry 1 in list file /etc/apt/sources.list.d/nvhpc.list (absolute Suite Component)
E: The list of sources could not be read.
E: Malformed entry 1 in list file /etc/apt/sources.list.d/nvhpc.list (absolute Suite Component)
E: The list of sources could not be read.


In [30]:
%%writefile vec_add_openacc.cu
#include <stdio.h>
#include <stdlib.h>

int main(void) {
    int n = 1 << 20;          // 1M elements
    size_t size = n * sizeof(float);

    float *a = (float*) malloc(size);
    float *b = (float*) malloc(size);
    float *c = (float*) malloc(size);

    for (int i = 0; i < n; ++i) {
        a[i] = 1.0f;
        b[i] = 2.0f;
    }

    // Offload this loop to GPU with OpenACC
    #pragma acc parallel loop copyin(a[0:n], b[0:n]) copyout(c[0:n])
    for (int i = 0; i < n; ++i) {
        c[i] = a[i] + b[i];
    }

    printf("c[0] = %f\n", c[0]);
    printf("c[n-1] = %f\n", c[n-1]);

    free(a);
    free(b);
    free(c);
    return 0;
}


Overwriting vec_add_openacc.cu


In [31]:
!nvc -acc -Minfo=accel vec_add_openacc.c -o vec_add_openacc
!./vec_add_openacc

/bin/bash: line 1: nvc: command not found
/bin/bash: line 1: ./vec_add_openacc: No such file or directory


In [32]:
!wget https://developer.download.nvidia.com/hpc-sdk/24.5/nvhpc_2024_245_Linux_x86_64_cuda_12.4.tar.gz
!tar xpzf nvhpc_2024_245_Linux_x86_64_cuda_12.4.tar.gz
!nvhpc_2024_245_Linux_x86_64_cuda_12.4/install
continue

--2025-11-17 08:46:12--  https://developer.download.nvidia.com/hpc-sdk/24.5/nvhpc_2024_245_Linux_x86_64_cuda_12.4.tar.gz
Resolving developer.download.nvidia.com (developer.download.nvidia.com)... 23.15.241.59, 23.15.241.8, 23.15.241.91, ...
Connecting to developer.download.nvidia.com (developer.download.nvidia.com)|23.15.241.59|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6769031775 (6.3G) [application/x-gzip]
Saving to: ‘nvhpc_2024_245_Linux_x86_64_cuda_12.4.tar.gz’

nvhpc_2024_245_Linu 100%[===================>]   6.30G  12.4MB/s    in 9m 0s   

2025-11-17 08:55:12 (12.0 MB/s) - ‘nvhpc_2024_245_Linux_x86_64_cuda_12.4.tar.gz’ saved [6769031775/6769031775]


Welcome to the NVIDIA HPC SDK Linux installer!

You are installing NVIDIA HPC SDK 2024 version 24.5 for Linux_x86_64.
Please note that all Trademarks and Marks are the properties
of their respective owners.

Press enter to continue...
^C


In [33]:
%cd /content/nvhpc_2024_245_Linux_x86_64_cuda_12.4

!NVHPC_SILENT=true \
  NVHPC_INSTALL_DIR=/opt/nvidia/hpc_sdk \
  NVHPC_INSTALL_TYPE=single \
  NVHPC_DEFAULT_CUDA=12.4 \
  ./install

/content/nvhpc_2024_245_Linux_x86_64_cuda_12.4
generating environment modules for NV HPC SDK 24.5 ... done.


In [34]:
import os

NVHPC = "/opt/nvidia/hpc_sdk"
NVARCH = "Linux_x86_64"
VERSION = "24.5"

os.environ["PATH"] = f"{NVHPC}/{NVARCH}/{VERSION}/compilers/bin:" + os.environ["PATH"]

!nvc -V



nvc 24.5-1 64-bit target on x86-64 Linux -tp skylake-avx512 
NVIDIA Compilers and Tools
Copyright (c) 2024, NVIDIA CORPORATION & AFFILIATES.  All rights reserved.


In [42]:
%%writefile vec_add_openacc.c
#include <stdio.h>
#include <stdlib.h>

int main(void) {
    int n = 1 << 20;          // 1M elements
    size_t size = n * sizeof(float);

    float *a = (float*) malloc(size);
    float *b = (float*) malloc(size);
    float *c = (float*) malloc(size);

    for (int i = 0; i < n; ++i) {
        a[i] = 1.0f;
        b[i] = 2.0f;
    }

    // Offload this loop to GPU with OpenACC
    #pragma acc parallel loop copyin(a[0:n], b[0:n]) copyout(c[0:n])
    for (int i = 0; i < n; ++i) {
        c[i] = a[i] + b[i];
    }

    printf("c[0] = %f\n", c[0]);
    printf("c[n-1] = %f\n", c[n-1]);

    free(a);
    free(b);
    free(c);
    return 0;
}

Writing vec_add_openacc.c


In [43]:
!nvc -acc -Minfo=accel vec_add_openacc.c -o vec_add_openacc
!./vec_add_openacc

main:
     15, Generating copyin(a[:n]) [if not already present]
         Generating copyout(c[:n]) [if not already present]
         Generating copyin(b[:n]) [if not already present]
         Generating implicit firstprivate(n,i)
         Generating NVIDIA GPU code
         19, #pragma acc loop gang, vector(128) /* blockIdx.x threadIdx.x */
c[0] = 3.000000
c[n-1] = 3.000000


In [51]:
%%writefile jacobi_openacc.f90

program jacobi_openacc
    implicit none
    integer, parameter :: nx = 512, ny = 512
    integer, parameter :: maxiter = 10000
    real,    parameter :: tol = 1.0e-4

    real, dimension(nx, ny) :: u, unew
    integer :: i, j, iter
    real    :: err

    ! Initialize the array
    u    = 0.0
    unew = 0.0

    ! Example: set top boundary to 1.0, others 0.0
    do i = 1, nx
        u(i, ny) = 1.0
        unew(i, ny) = 1.0
    end do

    print *, "Starting Jacobi iterations on GPU with OpenACC..."

    ! Put data on device once and keep it there
    !$acc data copy(u), create(unew)
    do iter = 1, maxiter

        err = 0.0

        ! Jacobi update on interior points
        !$acc parallel loop collapse(2) reduction(max:err)
        do j = 2, ny-1
            do i = 2, nx-1
                unew(i,j) = 0.25 * ( u(i+1,j) + u(i-1,j) + &
                                     u(i,j+1) + u(i,j-1) )

                err = max(err, abs(unew(i,j) - u(i,j)))
            end do
        end do
        !$acc end parallel loop

        ! Copy new values back into u for next iteration
        !$acc parallel loop collapse(2)
        do j = 2, ny-1
            do i = 2, nx-1
                u(i,j) = unew(i,j)
            end do
        end do
        !$acc end parallel loop

        if (mod(iter, 100) == 0) then
            print *, "Iter = ", iter, "  err = ", err
        end if

        if (err < tol) then
            print *, "Converged at iter = ", iter, " with err = ", err
            exit
        end if

    end do
    !$acc end data

    print *, "Final center value u(nx/2, ny/2) = ", u(nx/2, ny/2)

end program jacobi_openacc


Overwriting jacobi_openacc.f90


In [52]:
!nvfortran -acc -Minfo=accel jacobi_openacc.f90 -o jacobi_openacc
!./jacobi_openacc

jacobi_openacc:
     25, Generating copy(u(:,:)) [if not already present]
         Generating create(unew(:,:)) [if not already present]
     31, Generating implicit copy(err) [if not already present]
         Generating NVIDIA GPU code
         32, !$acc loop gang, vector(128) collapse(2) ! blockidx%x threadidx%x
             Generating reduction(max:err)
         33,   ! blockidx%x threadidx%x collapsed
     43, Generating NVIDIA GPU code
         44, !$acc loop gang, vector(128) collapse(2) ! blockidx%x threadidx%x
         45,   ! blockidx%x threadidx%x collapsed
 Starting Jacobi iterations on GPU with OpenACC...
 Iter =           100   err =    2.4214089E-03
 Iter =           200   err =    1.2103617E-03
 Iter =           300   err =    8.0645084E-04
 Iter =           400   err =    6.0504675E-04
 Iter =           500   err =    4.8398972E-04
 Iter =           600   err =    4.0322542E-04
 Iter =           700   err =    3.4567714E-04
 Iter =           800   err =    3.0252337E-04